<h1 style='color: green; font-size: 30px; font-weight: bold;'>Data Science - Regressão Logística: de baixo dos panos</h1>

# <font color='black' style='font-size: 24px;'>1.1 Conhecendo o Dataset</font>
<hr style='border: 2px solid black;'>

## Importando o numpy e pandas

In [1]:
import numpy as np
import pandas as pd

## O Dataset e o Projeto
<hr>

### Descrição:
<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>O mercado imobiliário vem sendo objeto de diversos estudos e pesquisas nos últimos tempos. A crise financeira que afeta a economia tem afetado significativamente os investimentos e ganhos advindos deste setor. Este cenário incentiva o aumento do interesse por estudos de previsão de demanda baseados em características deste mercado, dos imóveis e do entorno destes imóveis.</p>

<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>Neste contexto o objetivo principal do nosso projeto é desenvolver um sistema de avaliação imobiliária utilizando a metodologia de regressões lineares que é uma das técnicas de machine learning.</p>

<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>Nosso dataset é uma amostra aleatória de tamanho 5000 de imóveis disponíveis para venda no município do Rio de Janeiro.</p>

### Dados:
<ul style='font-size: 15px; line-height: 2; text-align: justify;'>
    <li><b>Valor</b> - Valor (R$) de oferta do imóvel;</li>
    <li><b>Area</b> - Área do imóvel em m².</li>
    <li><b>Dist_Praia</b> - Distância do imóvel até a praia (km) (em linha reta).</li>
    <li><b>Dist_Farmacia</b> - Distância do imóvel até a farmácia mais próxima (km) (em linha reta).</li>
    <li><b>Vale_a_pena_comprar</b> - Valor booleano indicando se vale a pena comprar este imóvel.</li>
</ul>

In [2]:
dados = pd.read_csv('dados_classificacao_multivariavel.csv')

In [3]:
dados.head()

,Valor,Area,Dist_Praia,Dist_Farmacia,vale_a_pena_comprar
0,4600000,280,0.240925,0.793637,1
1,900000,208,0.904136,0.134494,1
2,2550000,170,0.059525,0.423318,1
3,550000,100,2.883181,0.525064,0
4,2200000,164,0.239758,0.192374,1


In [4]:
valor = np.log(dados['Valor'])
area = np.log(dados['Area'])
dist_Praia = np.log(dados['Dist_Praia'] + 1)
dist_Farmacia = np.log(dados['Dist_Farmacia'] + 1)
vale_a_pena_comprar = dados['vale_a_pena_comprar']

X = np.array([valor, area, dist_Praia, dist_Farmacia]).T
y = vale_a_pena_comprar

# Regressão logística multivariável

Agora nosso problema se trata de uma classificação. Quero saber se vale a pena comprar um determinado imóvel. Para isto, vamos usar uma curva logística no lugar de uma linha. A função logística é uma função que varia entre 0 e 1, e é contínuo em todo seu domínio, definida da seguinte forma:

$$
y = \frac{1}{1+e^{-x}}
$$

Podemos definir isto a partir da notação matricial:

$$
y = \frac{1}{1+e^{-X\bullet \theta}}
$$

Onde $\theta$ são os parâmetros que devem ser otimizados e X são os atributos da amostra em questão.

Desta forma, nosso modelo irá retornar a probabilidade de nossa amostra ser da classe em questão.

In [5]:
def prever_prob(theta, X):
    z = np.dot(X, theta)
    return 1 / (1 + np.exp(-z))

E para classificar nossaas amostras, devemos simplesmente checar se a probabilidade está acima de um limiar desejado (geralmente 0.5).

In [6]:
def classificar(y, limiar=0.5):
    return (y > limiar) * 1

Agora, vamos a função de custo. Usaremos a função da entropia cruzada.

$$
Entropia Cruzada = \frac{1}{N}\sum_{i=1}^N(-y_i*log(y^p_i) - (1 - y_i)*log(1 - y^p_i))
$$

In [7]:
def entropia_cruzada(previstos, y):
    return (-y * np.log(previstos) - (1 - y) * np.log(1 - previstos)).mean()

Devemos minimizar nossa função de custo, para isto usamos o gradiente descendente. Nele, usamos a derivada de nossa função de custo, definida como:

$$
EntropiaCruzada' = \frac{1}{N}X^T\bullet(y^p - y)
$$

In [8]:
def gradienteDescendente(theta, X, y, alpha):
    previsto = prever_prob(theta, X)
    erro = previsto - y
    
    gradiente = np.dot(X.T, erro) / (len(X))

    theta -= alpha*gradiente

## Otimizando os parâmetros
O método c_ do numpy vai apenas colocar mais um atributo em todas amostras de nosso conjunto. Este ultimo atributo será sempre unitário, e será responsável pela variável independente $\theta_0$.

Após isto, vamos criar nossos pesos de forma aleatória e executar a nossa otimização.

In [9]:
X = np.c_[np.ones(X.shape[0]), X]

np.random.seed(42)

theta = np.random.rand(X.shape[1])

for i in range(7000):
    previsto = prever_prob(theta, X)
    custo = entropia_cruzada(previsto, y)
    
    if i % 1000 == 0:
        print(custo)
    
    gradienteDescendente(theta, X, y, 0.1)

8.640393601108922
0.3068509524080465
0.29882549538137093
0.2983581074563659
0.2980646560845418
0.29780855042822835
0.2975664466572869


### Vale a pena comprar um apartamento de 72m², a 500 m da praia e 100 metros da farmácia por R$ 850.000?

In [10]:
minha_area = np.log(72)
minha_dist_praia = np.log(0.5) + 1
minha_dist_farmacia = np.log(0.1) + 1
meu_valor = np.log(850000)

minha_casa = [1, minha_area, minha_dist_praia, minha_dist_farmacia, meu_valor]

classificar(prever_prob(theta, minha_casa), 0.5)

1

### Para checar a eficiencia do modelo, vamos usar a acurácia.

In [11]:
classificados = classificar(prever_prob(theta, X))

In [12]:
(classificados == y).mean()

0.8934

# Comparando com o Sk-learn

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
lr = LogisticRegression(random_state=42)

In [15]:
lr.fit(X[:,1:], y)

LogisticRegression(random_state=42)

In [16]:
(lr.predict(X[:,1:]) == y).mean()

0.8904

## Divisão em conjuntos de treinamento e teste

In [17]:
import random

In [18]:
def divisao_treinamento_teste(X, y, porcentagem_teste, random_seed=42):
    """Faz a separação dos arrays conforme a indicação do percentual que deve ser um numero < 1"""
    """Retorna np.array(X_treino), np.array(X_teste), np.array(y_treino), np.array(y_teste)"""
    random.seed(random_seed)
    
    X_teste, y_teste = [], []
    
    X_treino = list(X)
    y_treino = list(y)
    
    tam_y = porcentagem_teste * len(y)
    
    while len(y_teste) < tam_y:
        index = random.randrange(len(X_treino))
        X_teste.append(X_treino.pop(index))
        y_teste.append(y_treino.pop(index))
        
    return np.array(X_treino), np.array(X_teste), np.array(y_treino), np.array(y_teste)   


## Refazendo nosso modelo

In [19]:
X_treino, X_teste, y_treino, y_teste = divisao_treinamento_teste(X[:,1:], y, 0.2)

X_treino = np.c_[np.ones(X_treino.shape[0]), X_treino]
theta = np.random.rand(X_treino.shape[1])

for i in range(7000):
    previsto = prever_prob(theta, X_treino)
    custo = entropia_cruzada(previsto, y_treino)
    
    if i % 1000 == 0:
        print(custo)
    
    gradienteDescendente(theta, X_treino, y_treino, 0.1)

3.028514164989356
0.31792324246109205
0.3083916430654404
0.30571640814580725
0.3051437988826891
0.30486037452781517
0.3045890579056707


## Avaliando o conjunto de treinamento

In [20]:
classificados = classificar(prever_prob(theta, X_treino))

print((classificados == y_treino).mean())

0.891


## Avaliando o conjunto de teste

In [21]:
X_teste = np.c_[np.ones(X_teste.shape[0]), X_teste]

classificados = classificar(prever_prob(theta, X_teste))
print((classificados == y_teste).mean())

0.9
